In [76]:
from pprint import pprint
import pandas as pd
from markets.markets import Markets
from database.database import Database
from settings.settings import Settings

In [2]:
db = Database()

Connecting to database...OK


In [3]:
db.get_exchanges()
db.get_tokens()
db.get_pairs()

In [4]:
df_exchangs = db.df_exchanges
df_tokens = db.df_tokens
df_pairs = db.df_pairs[['pair', 'exchange']]

In [5]:
markets = Markets(db)

CCXT version: 1.15.27


In [6]:
#df_pairs.sort_values(['pair','exchange']).set_index('pair')
pairs = pd.DataFrame(df_pairs.groupby(['pair'])['exchange'].apply(list)).exchange.to_dict()

In [7]:
pairs

{'ADA/BTC': ['binance', 'bittrex', 'hitbtc2', 'cryptopia'],
 'ADA/DOGE': ['cryptopia'],
 'ADA/ETH': ['binance', 'bittrex', 'hitbtc2'],
 'ADA/LTC': ['cryptopia'],
 'ADA/USDT': ['binance', 'bittrex', 'hitbtc2'],
 'BCH/BTC': ['kraken',
  'poloniex',
  'yobit',
  'binance',
  'okex',
  'bittrex',
  'exmo',
  'hitbtc2',
  'cryptopia'],
 'BCH/DOGE': ['yobit', 'cryptopia'],
 'BCH/ETH': ['poloniex',
  'yobit',
  'binance',
  'okex',
  'bittrex',
  'exmo',
  'hitbtc2'],
 'BCH/LTC': ['cryptopia'],
 'BCH/USD': ['kraken', 'yobit', 'okex', 'exmo'],
 'BCH/USDT': ['poloniex',
  'binance',
  'okex',
  'bittrex',
  'hitbtc2',
  'cryptopia'],
 'BTC/USD': ['kraken', 'yobit', 'okex', 'bittrex', 'exmo'],
 'BTC/USDT': ['poloniex',
  'binance',
  'okex',
  'bittrex',
  'exmo',
  'hitbtc2',
  'cryptopia'],
 'BTG/BTC': ['yobit', 'binance', 'okex', 'bittrex', 'exmo', 'hitbtc2'],
 'BTG/DOGE': ['yobit'],
 'BTG/ETH': ['yobit', 'binance', 'bittrex', 'hitbtc2'],
 'BTG/USD': ['yobit', 'okex', 'exmo'],
 'BTG/USDT': ['

In [8]:
exchanges = df_pairs.groupby(['exchange'])['pair'].apply(list).to_dict()

In [9]:
exchanges['hitbtc2'][3:5]

['BCH/BTC', 'BCH/ETH']

In [10]:
print(exchanges['exmo'])

['BCH/BTC', 'BCH/ETH', 'BCH/USD', 'BTC/USD', 'BTC/USDT', 'BTG/BTC', 'BTG/USD', 'DASH/BTC', 'DASH/USD', 'DOGE/BTC', 'EOS/BTC', 'EOS/USD', 'ETC/BTC', 'ETC/USD', 'ETH/BTC', 'ETH/LTC', 'ETH/USD', 'ETH/USDT', 'LTC/BTC', 'LTC/USD', 'USDT/USD', 'XLM/BTC', 'XLM/USD', 'XMR/BTC', 'XMR/USD', 'XRP/BTC', 'XRP/USD', 'ZEC/BTC', 'ZEC/USD']


# Fetch historic data from exchanges

In [11]:
exchanges = df_exchangs.id.tolist()
exchanges

['kraken',
 'poloniex',
 'yobit',
 'binance',
 'okex',
 'bittrex',
 'exmo',
 'hitbtc2',
 'cryptopia']

In [172]:
pair = "ETH/USD"
pair_alt = "ETH/USDT" # alternative representation
history = {}

# loading pairs from exchanges
for exchange in exchanges:
    history[exchange] = markets.fetch_trades(exchange, pair, limit=100)
    if history[exchange] == None:
        history[exchange] = markets.fetch_trades(exchange, pair_alt, limit=100)
    elif history[exchange] == None:
        del(history[exchange])

poloniex No market symbol ETH/USD
binance No market symbol ETH/USD
hitbtc2 No market symbol ETH/USD
cryptopia No market symbol ETH/USD


## Kraken and Cryptopia has no IDs so we need to generate them by ourselves:

In [173]:
infos = {}
for ex in exchanges:
    infos[ex] = history[ex][0]['id']
    
pprint(infos)

{'binance': '28856849',
 'bittrex': '1408060',
 'cryptopia': None,
 'exmo': '65012105',
 'hitbtc2': '335014018',
 'kraken': None,
 'okex': '1150815922193419',
 'poloniex': '10123268',
 'yobit': '201211101'}


## Check whether an Exchange have full info on transaction side and type

In [174]:
for ex in exchanges:
    print("\n============\n  " + ex + "\n============")
    pprint(history[ex][0])


  kraken
{'amount': 0.06050148,
 'cost': 28.0315457136,
 'datetime': '2018-07-22T19:57:02.139Z',
 'fee': None,
 'id': None,
 'info': ['463.32000', '0.06050148', 1532289422.139, 'b', 'l', ''],
 'order': None,
 'price': 463.32,
 'side': 'buy',
 'symbol': 'ETH/USD',
 'timestamp': 1532289422139,
 'type': 'limit'}

  poloniex
{'amount': 0.50178427,
 'cost': 233.85687475,
 'datetime': '2018-07-22T20:38:44.000Z',
 'fee': None,
 'id': '10123268',
 'info': {'amount': '0.50178427',
          'date': '2018-07-22 20:38:44',
          'globalTradeID': 383626542,
          'rate': '466.05062920',
          'total': '233.85687475',
          'tradeID': 10123268,
          'type': 'sell'},
 'order': None,
 'price': 466.0506292,
 'side': 'sell',
 'symbol': 'ETH/USDT',
 'timestamp': 1532291924000,
 'type': 'limit'}

  yobit
{'amount': 0.02420813,
 'datetime': '2018-07-22T20:03:39.000Z',
 'fee': {'cost': 0.022375898829684,
         'currency': 'USD',
         'rate': 0.002,
         'type': 'taker'},
 '

## Compare prices

In [180]:
prices = [{'exchange':ex, 'price':history[ex][-1]['price']} for ex in exchanges]

In [181]:
df = pd.DataFrame(prices)
df

,exchange,price
0,kraken,463.760000
1,poloniex,459.297498
2,yobit,461.109978
3,binance,457.520000
4,okex,458.006000
5,bittrex,466.787000
6,exmo,459.392000
7,hitbtc2,459.120000
8,cryptopia,466.854417


In [182]:
# maximum spread
df_minmax = df.iloc[[df.price.idxmin(),df.price.idxmax()]]
df_minmax

,exchange,price
3,binance,457.520000
8,cryptopia,466.854417


In [191]:
# calculate diff and percentage
print(f"Difference between min and max = "
      f"{df_minmax.iloc[1].price - df_minmax.iloc[0].price} {pair.split('/')[1]}"
      f" or {round((df_minmax.iloc[1].price / df_minmax.iloc[0].price - 1)*100, 2)}%")

Difference between min and max = 9.33441732 USD or 2.04%
